In [1]:
# from zhipuai import ZhipuAI
# import csv

import json

# 读取JSON文件
with open('fudan.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
# 输出读取的数据
# print(data[0])
print(len(data))

579


In [2]:
import dashscope
import random
from http import HTTPStatus
# 建议dashscope SDK 的版本 >= 1.14.0
from dashscope import Generation

dashscope.api_key="register-your-key"

newData = []

for index, item in enumerate(data):

    if index > 117:
        # for item in data:
        prompt = ("入组标准：\n" + item[
            "InclusionCriteria"] + "根据以上内容生成每个小点对应的英文问题，向患者提问，问题数量适中，必须要有判断题也要有填空题。直接生成不重复的问题，以json形式[{\"question\":\"\",\"type\":\"\",\"option\":[]}]输出。(注：question为问题，type是问题的类型，填空题type为1，判断题type为2，option为选项，当type为1时option为[])。英文输出，只输出JSON不要生成其他内容"
                  + "例如：入组标准：1、60 years or more at time of transplantation. 2、Acute leukemia. 3、Myelodysplastic syndrome. 4、Myeloproliferative neoplasm.生成每个小点对应的问题，向患者提问，必须要有判断题也要有填空题。直接生成问题，以json形式{\"question\":\"\",\"type\":\"\",\"option\":[]}输出。(注：填空题type为1，判断题type为2，question为问题，type是问题的类型，option为选项)英文输出。"
                  + "生成结果：[{\"question\":\"What is your age at the time of transplantation?\",\"type\":1,\"option\":[]},{\"question\":\"Do you have acute leukemia?\",\"type\":2,\"option\":[\"Yes\",\"No\"]},{\"question\":\"Have you been diagnosed with myelodysplastic syndrome?\",\"type\":2,\"option\":[\"Yes\",\"No\"]},{\"question\":\"Have you been diagnosed with myeloproliferative neoplasm?\",\"type\":2,\"option\":[\"Yes\",\"No\"]}]"
                  )

        messages = [{'role': 'system', 'content': 'You are a helpful assistant.'},
                    {'role': 'user', 'content': prompt}]
        response = dashscope.Generation.call(
        model='llama3-70b-instruct',
        messages=messages,
        result_format='message')  # set the result to be "message" format.
        # response = dashscope.Generation.call(model="baichuan2-7b-chat-v1",
        #                            messages=messages,
        #                            # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
        #                            # seed=random.randint(1, 10000),
        #                            # temperature=0.8,
        #                            # top_p=0.8,
        #                            # top_k=50,
        #                            # 将输出设置为"message"格式
        #                            result_format='message')
        if response.status_code == HTTPStatus.OK:
            print(response.output.choices[0].message.content)
            itemData = {
            "nct_id": item["NCTID"],
            "study_title": item["Study_Title"],
            "inclusion_criteria": item["InclusionCriteria"],
            "inclusion_criteria_questions": response.output.choices[0].message.content
            }
            newData.append(itemData)
            print(index)
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))



Here is the generated JSON output:

[{"question":"Have you been diagnosed with platinum-sensitive recurrent ovarian cancer?","type":2,"option":["Yes","No"]},
{"question":"How long was the recurrence interval before the last platinum-based chemotherapy?","type":1,"option":[]},
{"question":"Did you experience progression within 3 months during the use of bevacizumab?","type":2,"option":["Yes","No"]},
{"question":"Did you experience progression within 12 months during the use of PARP inhibitors?","type":2,"option":["Yes","No"]},
{"question":"What was your response to the last platinum-based chemotherapy?","type":1,"option":[]},
{"question":"What is your current ECOG performance status?","type":1,"option":[]},
{"question":"Do you have a life expectancy of at least 3 months?","type":2,"option":["Yes","No"]},
{"question":"What is your current creatinine clearance rate (mL/min)?","type":1,"option":[]},
{"question":"Do you voluntarily agree to participate in this study?","type":2,"option":["Ye

In [3]:
len(newData)

461

In [4]:
import csv
headers = ["nct_id", "study_title", "inclusion_criteria", "inclusion_criteria_questions"]
# 写入CSV文件
with open('FuDanData_llama3-70b-instruct117-578.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()  # 写入头信息
    for item in newData:
        writer.writerow(item)  # 写入数据行